In [1]:
pip install countryinfo

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Importing all my stuff
import pandas as pd
import numpy as np

from dataprep.clean import clean_country
import fuzzywuzzy
from fuzzywuzzy import process, fuzz
import requests
import datetime as dt
import time
import pycountry
from countryinfo import CountryInfo
import re
from subprocess import check_output
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from dataprep.clean import clean_date
weather_api_key = "a8035cf966e04b4612afafe30e26e55a"
geoapify_key = "a2eb724159ed4830944bd40a55611baa"

In [9]:
#Creating a function that will parse through countries and determine if they are in the northern or southern hemisphere

def country_hem(col):
    hem_lst = []
    for row in col:
        try:
            country = CountryInfo(row)
        except AttributeError:
            row = 'x'
            print(row)
   
        try:
            pos = country.latlng()
        except KeyError:
            pos = (0,0)
    
        if pos[0] > 0:
            temp_row = 1
            print(temp_row)
        elif pos[0] < 0:
            temp_row = 0
            print(temp_row)
        else:
            temp_row = 3
            print(temp_row)
    
        hem_lst.append(temp_row)
    
    return hem_lst   

# def country_hem(col):
#     hem_lst = []
#     for row in col:
#         try:
#             country = CountryInfo(row)
#         except AttributeError:
#             row = 'x'
#         try:
#             pos = country.latlng()
#         except KeyError:
#             pos = (0,0)
    
#         if pos[0] > 0:
#             temp_row = 1
#         elif pos[0] < 0:
#             temp_row = 0
#         else:
#             temp_row = 3
#         hem_lst.append(temp_row)
#     return hem_lst 

In [4]:
#Bringing in the file
file = "attacks.csv"
#Readin the file
df = pd.read_csv(file, encoding="cp1252")
#Renaming the columns to make them better for me
df.columns = [i.lower().replace(" ", "_") for i in df.columns]
df = df.rename(columns = {"fatal_(y/n)" : "fatal", 'investigator_or_source' : "investigator", 'sex_' : "sex"
                         , 'species_' : "species"})
#Getting rid of the rows where case number is null
df = df[~df["case_number"].isnull()] 
#Getting rid of the row where case number is xx
df = df.loc[df["case_number"]  != "xx", :]
#Getting rid of the row where the case number is 0
df = df.loc[df["case_number"]  != "0", :]
#Dropping useless columns
df = df.drop(columns=['pdf', 'href_formula', 'href', 'case_number.1', 'case_number.2', 'original_order', 'unnamed:_22', 'unnamed:_23', 'case_number']) 
df = df.reset_index()

In [5]:
#Cleaning up countries. This creates a new columne called country_clean (more on that shortly)
df = clean_country(df, "country", fuzzy_dist=1)
#Getting rid of the original dirty country column
df = df.drop(columns=['country'])
#Renaming country_clean
df = df.rename(columns = {"country_clean" : "country"})

C:\Users\Grant\anaconda3\envs\dev\lib\site-packages\dask\dataframe\core.py:7234: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


  0%|                                                                                            | 0/8 [00:00<…

Country Cleaning Report:
	6091 values cleaned (96.67%)
	155 values unable to be parsed (2.46%), set to NaN
Result contains 6096 (96.75%) values in the correct format and 205 null values (3.25%)


In [6]:
#This will create a new column called month. Which is created from the date column.
month_list = []
for row in df['date']:
    month_row = ''.join(re.findall('\-[A-Za-z]{3}\-',row)).lower()
    month_row = re.sub('\-','',month_row)
    if month_row == '':
        month_row = np.nan
    month_list.append(month_row)
df['month'] = month_list

In [7]:
#Cleaning up incorrect month data.
df['month'] = df['month'].replace({'marmar': 'mar', 'augaug': 'aug', 'sepsep': 'sep', 'janjan': 'jan', 'novnov': 'nov'})
df['month'].fillna('Unknown', inplace=True)
df.head()

,index,date,year,type,area,location,activity,name,sex,age,injury,fatal,time,species,investigator,country,month
0,0,25-Jun-18,2018.0,Boating,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",United States,jun
1,1,18-Jun-18,2018.0,Unprovoked,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",United States,jun
2,2,9-Jun-18,2018.0,Invalid,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",United States,jun
3,3,8-Jun-18,2018.0,Unprovoked,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",Australia,jun
4,4,4-Jun-18,2018.0,Provoked,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,Mexico,jun


In [10]:
#Creating a new column that utilizes the country_hem function from above.
df['hemisphere'] = country_hem(df['country']) 
df.head(2)

1
1
1
0
1
0
0
1
1
1
0
1
1
x
1
0
0
0
1
1
0
0
0
0
1
0
1
1
0
0
0
3
0
0
3
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
3
1
1
1
1
1
x
1
0
1
0
0
1
0
1
1
0
0
1
1
1
0
1
0
1
1
1
1
1
0
0
0
1
0
0
0
0
1
1
1
1
0
1
1
0
1
3
1
1
1
1
1
1
1
3
3
3
1
1
1
1
1
1
1
0
1
1
1
0
0
1
1
1
1
0
1
0
3
0
0
1
1
1
3
1
1
0
1
0
1
1
1
0
1
0
0
1
3
1
1
0
1
1
1
0
1
1
1
1
1
0
1
1
0
1
1
3
0
0
0
0
1
0
0
1
3
0
1
0
0
1
0
0
1
0
1
0
0
0
0
1
0
0
0
1
1
1
0
1
0
1
1
0
1
1
1
0
1
1
1
1
0
0
0
1
1
0
1
0
1
1
1
1
1
0
1
3
1
1
1
1
0
1
0
1
0
3
0
1
1
1
1
1
3
1
1
1
1
1
0
1
1
1
0
1
1
1
0
1
1
1
1
0
1
0
0
0
1
1
1
1
1
1
1
1
0
0
1
0
0
0
0
1
0
3
1
1
1
0
1
0
3
1
1
0
1
0
0
0
0
1
1
0
0
0
0
0
1
1
1
1
1
0
0
0
0
1
1
0
0
1
0
3
0
0
1
1
1
0
1
1
1
0
0
1
0
1
1
0
1
1
1
1
1
0
0
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
0
1
1
0
1
1
1
0
1
1
1
1
1
0
1
0
1
1
0
1
0
0
0
1
1
0
3
1
0
0
1
0
1
1
0
1
0
1
0
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
0
1
0
0
1
1
0
0
1
0
0
1
1
0
0
1
1
0
0
1
0
x
0
0
0
0
0
0
1
0
0
1
0
0
0
1
3
0
1
0
0
1
0
1
0
1
0
1
0
0
0
1
1
1
0
1
1
0
1
1
1
1
0
1
1
0
1
1
1
1
1
1
0


,index,date,year,type,area,location,activity,name,sex,age,injury,fatal,time,species,investigator,country,month,hemisphere
0,0,25-Jun-18,2018.0,Boating,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",United States,jun,1
1,1,18-Jun-18,2018.0,Unprovoked,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",United States,jun,1


In [11]:
#Creating dictionaries for seasons for northern and southern hemispheres.
north_seasons = {'spring': ['mar', 'apr', 'may'],
                 'summer': ['jun', 'jul', 'aug'],
                 'fall': ['sep', 'oct', 'nov'],
                 'winter': ['dec', 'jan', 'feb']}

south_seasons = {'spring': ['sep', 'oct', 'nov'],
                 'summer': ['dec', 'jan', 'feb'],
                 'fall': ['mar', 'apr', 'may'],
                 'winter': ['jun', 'jul', 'aug']}

In [12]:
#Running through the each row to apply season with the dictionaries created perviously.
seasonz = []
for row in df.index:
    if df.at[row, 'hemisphere'] == 1:
        if df.at[row, 'month'] in north_seasons['spring']:
            seasonz.append('spring')
        elif df.at[row, 'month'] in north_seasons['summer']:
            seasonz.append('summer')
        elif df.at[row, 'month'] in north_seasons['fall']:
            seasonz.append('fall')
        elif df.at[row, 'month'] in north_seasons['winter']:
            seasonz.append('winter')
        else:
            seasonz.append('Unknown')
    elif df.at[row, 'hemisphere'] == 0:
        if df.at[row, 'month'] in south_seasons['spring']:
            seasonz.append('spring')
        elif df.at[row, 'month'] in south_seasons['summer']:
            seasonz.append('summer')
        elif df.at[row, 'month'] in south_seasons['fall']:
            seasonz.append('fall')
        elif df.at[row, 'month'] in south_seasons['winter']:
            seasonz.append('winter')
        else:
            seasonz.append('Unknown')
    else:
        seasonz.append('Unknown')
df['seasons'] = seasonz

In [13]:
df.head()

,index,date,year,type,area,location,activity,name,sex,age,injury,fatal,time,species,investigator,country,month,hemisphere,seasons
0,0,25-Jun-18,2018.0,Boating,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",United States,jun,1,summer
1,1,18-Jun-18,2018.0,Unprovoked,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",United States,jun,1,summer
2,2,9-Jun-18,2018.0,Invalid,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",United States,jun,1,summer
3,3,8-Jun-18,2018.0,Unprovoked,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",Australia,jun,0,winter
4,4,4-Jun-18,2018.0,Provoked,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,Mexico,jun,1,summer
